In [48]:
import os
import pandas as pd
import psycopg2
from itertools import combinations


In [49]:
def calling_redshift():
    """
    This function connects to Amazon Redshift.
    """
    con = psycopg2.connect(dbname='', 
                           host='', 
                           port='',
                           user='',    
                           password='')
    return con

In [50]:
def retrieve_maid_for_segments(start_date, end_date, min_lat, min_long, max_lat, max_long, start_hour, end_hour, country_iso3, segment_file):
    """
    Retrieves the MAID for each segment and saves segments in separate files.
    """
    segments_df = pd.read_csv(segment_file, header=None, skiprows=1)
    segments = segments_df[0].tolist()

    cur_profile_table = f"cur_profile_segment_affinity_{country_iso3}"
    transdata_table = f"transdata_{country_iso3}"
    
    segment_placeholders = ','.join(['%s'] * len(segments))
    
    query = f"""
    SELECT DISTINCT
        data.segment AS name,       
        data.maid
    FROM
        {cur_profile_table} AS data
    JOIN (
        SELECT DISTINCT
            data.maid
        FROM {transdata_table} AS data
        WHERE (data.latitude > %s AND data.latitude < %s AND data.longitude > %s AND data.longitude < %s) 
            AND DATE(data.transaction_datetime) BETWEEN %s AND %s
            AND EXTRACT(HOUR FROM data.transaction_datetime) BETWEEN %s AND %s
    ) AS filtered_data ON data.maid = filtered_data.maid
    WHERE data.segment IN ({segment_placeholders})  -- Filter segments based on CSV file
    GROUP BY data.segment, data.maid;  -- Include data.maid in the GROUP BY clause
    """
    
    with calling_redshift() as con:
        cur = con.cursor()
        cur.execute(query, (min_lat, max_lat, min_long, max_long, start_date, end_date, start_hour, end_hour) + tuple(segments))
        df = pd.DataFrame(cur.fetchall(), columns=['name', 'maid'])

    print("Retrieved data:", df) 
    for segment_index, segment in enumerate(segments, start=1):
        segment_df = df[df['name'] == segment]
        segment_filename = f'/home/fazan123/MW/overlapping/New/{segment}.csv'
        segment_df.to_csv(segment_filename, index=False)

    return df


### In Below Cell Update Custom Hours and Run to Calculate the Overlapping Probailities between only 2 segments

In [51]:

# def calc_prob(df):
#     """
#     Calculates the probability of overlap between segments.
#     """
#     seg1 = []
#     seg2 = []
#     prob_l = []
#     seg_l = df['name'].unique()

#     for i in range(len(seg_l)):
#         tmp1 = df[df['name'] == seg_l[i]]
#         maid_list1 = set(tmp1['maid'])

#         for u in range(len(seg_l)):
#             if seg_l[u] != seg_l[i]:
#                 tmp2 = df[df['name'] == seg_l[u]]
#                 maid_list2 = set(tmp2['maid'])

#                 inter = maid_list1.intersection(maid_list2)

#                 try:
#                     prob = ((len(inter) / len(maid_list1)) + (len(inter) / len(maid_list2))) / 2
#                     seg1.append(seg_l[i])
#                     seg2.append(seg_l[u])
#                     prob_l.append(prob)
#                 except ZeroDivisionError:
#                     pass

#     final = pd.DataFrame(list(zip(seg1, seg2, prob_l)), columns=['segment1', 'segment2', 'Probability'])
#     return final

In [52]:
# max_lat = -6.2213
# max_long = 106.8132
# min_lat = -6.2303
# min_long = 106.8042
# start_date = '2024-02-22'
# end_date = '2024-02-23'
# start_hour = 0
# end_hour = 10
# country_iso3 = "IDN"
# segment_file = "/home/fazan123/MW/overlapping/sample_file/sample_segments.csv"

# df = retrieve_maid_for_segments(start_date, end_date, min_lat, min_long, max_lat, max_long, start_hour, end_hour, country_iso3, segment_file)
# prob_df = calc_prob(df)
# prob_df.to_csv(f'/home/fazan123/MW/overlapping/New/overlapping_segment_probabilities_in_{country_iso3}_from_{start_date}_to_{end_date}_between_{start_hour}_and_{end_hour}_for_custom_segments.csv', index=False)

### In Below Cell Update values  and Run to Calculate the Overlapping Probailities between all combinations of segments

In [53]:
def calc_prob_for_all_combinations(df, output_dir):
    """
    Calculates the probability of overlap between segments for combinations of segments and writes the results to a CSV file.
    """
    prob_data = []
    seg_l = df['name'].unique()

    for n_segments in range(2, len(seg_l) + 1):
        for combination in itertools.combinations(seg_l, n_segments):
            maid_sets = [set(df[df['name'] == segment]['maid']) for segment in combination]
            intersection = set.intersection(*maid_sets)
            try:
                prob = sum(len(intersection) / len(maid_set) for maid_set in maid_sets) / n_segments
                prob_data.append((','.join(combination), prob))
            except ZeroDivisionError:
                pass

    final = pd.DataFrame(prob_data, columns=['Segments', 'Overlap Percentage'])

    output_file = os.path.join(output_dir, 'overlap_probabilities_for_all_combinations_of_segments.csv')
    final.to_csv(output_file, index=False)
    print(f"Output saved to: {output_file}")


In [54]:
start_date = '2024-02-22'
end_date = '2024-02-23'
min_lat = -6.2303
min_long = 106.8042
max_lat = -6.2213
max_long = 106.8132
start_hour = 0
end_hour = 10
country_iso3 = "IDN"
segment_file = "/home/fazan123/MW/overlapping/sample_file/sample_segments.csv"
output_dir = "/home/fazan123/MW/overlapping/New"
df = retrieve_maid_for_segments(start_date, end_date, min_lat, min_long, max_lat, max_long, start_hour, end_hour, country_iso3, segment_file)
calc_prob_for_all_combinations(df, output_dir)

Retrieved data:                   name                                       maid
0      cat_fashionista  GAID_1e184477-6f02-4abb-af70-287d8e3df8b1
1      cat_fashionista  GAID_4c6b630c-8208-43ba-b87f-7460f85e3476
2      cat_fashionista  GAID_9615d3e8-c7ca-4d18-bbd2-d5c83f62b7bf
3      cat_fashionista  GAID_04e20c6e-3dc6-46f7-b4a3-922f25d6c50e
4      cat_fashionista  GAID_4b1697ce-0248-60f0-2436-8dc21f6a1874
...                ...                                        ...
2990  cat_white_collar  GAID_341e427b-4e15-488e-aa97-3d599f31f2a7
2991   cat_fashionista  GAID_49552864-4269-44d5-852f-921f7819a2b6
2992   cat_fashionista  GAID_6ce477df-2af5-4f64-be4a-fb80da7b1a67
2993   cat_fashionista  GAID_7a87a2b8-b6a0-4676-bf8b-729ea0a44bc0
2994   cat_fashionista  GAID_8f16b9bf-7aee-4bbc-aec3-676e2492f2bc

[2995 rows x 2 columns]
Output saved to: /home/fazan123/MW/overlapping/New/overlap_probabilities_for_all_combinations_of_segments.csv
